In [1]:
! pip install selenium
! pip install beautifulsoup4

In [53]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import time
from collections import defaultdict

In [2]:
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen as url 
import pickle
import json
import time
import multiprocessing
import pandas as pd 
import numpy as np
import re
import collections
import pickle

In [3]:
chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')  # Runs Chrome in headless mode
chrome_options.add_argument('--no-sandbox')  # Bypass OS security model
chrome_options.add_argument('--disable-dev-shm-usage')
chromedriver_path = "./chromedriver"

In [4]:
def class_extention_method(cls):
    def decorator(func):
        setattr(cls, func.__name__, func)
        return func
    return decorator

In [5]:
@class_extention_method(webdriver.chrome.webdriver.WebDriver)
def wait_elements_xpath(self, xpath, timeout = 5, verbose=True):
    try:
        return WebDriverWait(self,timeout).until(lambda d: d.find_elements_by_xpath(xpath))
    except:
        if verbose:
            print("find xpath timeout:",xpath)
        
@class_extention_method(webdriver.chrome.webdriver.WebDriver)
def wait_element_xpath(self, xpath, timeout = 5, verbose=True):
    try:
        return WebDriverWait(self,timeout).until(lambda d: d.find_element_by_xpath(xpath))
    except:
        if verbose:
            print("find xpath timeout:",xpath)

In [6]:
comment_info_xpath = "//div[@id='main']"
show_more_button_xpath = "//span[@class='more-button style-scope ytd-comment-renderer']"
number_of_comments = "//h2[@id='count']/yt-formatted-string[@class='count-text style-scope ytd-comments-header-renderer']"
# comment_total_amount_xpath = "//yt-formatted-string[@class='count-text style-scope ytd-comments-header-renderer']"

In [11]:
df = pd.read_excel('DataSource_80clips.xlsx')
# remove the "who" and sort the table with "comedian"
df = df.drop('Who?', axis=1)
df = df.sort_values('Comedian')
df.tail()

,Comedian,Title,Link of the transcript,Transcript (CONTENT!!!!),Video link,Unnamed: 6,note,Unnamed: 8,Unnamed: 9
27,Vir Das,Losing It (2018),https://scrapsfromtheloft.com/2018/12/15/vir-d...,Every time a woman in India wears something re...,https://www.youtube.com/watch?v=qtswR_SW8S0&ab...,NaN,NaN,NaN,NaN
17,Whitney Cummings,Marriage & Sex,https://scrapsfromtheloft.com/2017/12/25/whitn...,Love makes you think about things you never th...,https://www.youtube.com/watch?v=T6GstyTrLl0,NaN,NaN,NaN,NaN
14,Whitney Cummings,I'm Your Girlfriend: Women Paying,https://scrapsfromtheloft.com/2017/12/25/whitn...,We’re living in a fascinating time. We’re livi...,https://www.youtube.com/watch?v=A1OKxp91B2M,NaN,NaN,NaN,NaN
46,NaN,NaN,https://scrapsfromtheloft.com/2020/06/25/eric-...,NaN,https://www.youtube.com/watch?v=pr_D9-JGmsw&ab...,NaN,NaN,NaN,NaN
48,NaN,NaN,https://scrapsfromtheloft.com/2020/06/22/roy-w...,NaN,https://www.youtube.com/watch?v=mThkzqbcpU8&ab...,NaN,NaN,NaN,NaN


In [12]:
all_urls = df["Video link"]

In [13]:
len(all_urls)

80

In [17]:
web_list = []
with webdriver.Chrome(chromedriver_path,options=chrome_options) as driver:    
    for url in all_urls:
        all_comments_info = []        
        prev_offest = -1
        offset = 0
        
        print(f"\nUrl: {url}")
        driver.get(url)
        # Scroll until the number of comments shows up
        while driver.wait_elements_xpath(number_of_comments, timeout=1, verbose=False) == None:
            driver.execute_script("scrollBy(0, 50)")
            
        # Scarpe comments until all the comments are collected
        while offset != prev_offest:
            prev_offest = offset
            driver.execute_script("scrollBy(0, 1000)")
            time.sleep(2)
            offset = driver.execute_script("return window.pageYOffset;");
            all_comments_info = driver.wait_elements_xpath(comment_info_xpath)
            print("", end=".")
        # Get whole html for us to find sth later
        html = driver.page_source    
        whole_page = bs(html,"html.parser")
        web_list.append(whole_page)


Url: https://www.youtube.com/watch?v=gvSqKSkRFuM&ab_channel=HBO
..........................................
Url: https://www.youtube.com/watch?v=obphkbG5FIo&ab_channel=DryBarComedy
..........................
Url: https://www.youtube.com/watch?v=EmzPp33kl7o
............
Url: https://www.youtube.com/watch?v=c_KmV-hquvc&ab_channel=NetflixIsAJoke
...........
Url: https://www.youtube.com/watch?v=oTHpQjE1VQE&ab_channel=NetflixIsAJoke
.................................
Url: https://www.youtube.com/watch?v=lWA-xsAVzgE
.............
Url: https://www.youtube.com/watch?v=X8Ug3-_S174&ab_channel=NetflixIsAJoke
...........................................
Url: https://www.youtube.com/watch?v=io2d_cpoLDg&ab_channel=NetflixIsAJoke
.......................................
Url: https://www.youtube.com/watch?v=Oa6X6ufoiBM&ab_channel=GrettaVonPissflaps
.........................................
Url: https://www.youtube.com/watch?v=RRLf-NaMfLc&list=PLVXN0tFUddVTkbmqYUTS_OCkMF2ab-NpX&index=4&ab_channel=dncZTV
.

In [18]:
len(web_list)

80

In [29]:
# get clips title
def get_titles(web_list):
    clip_title = []
    for webpage in web_list:
        title = webpage.find('yt-formatted-string',{'class': 'style-scope ytd-video-primary-info-renderer'}).getText()
        clip_title.append(title)
    return clip_title

In [68]:
def html_to_list(web_list):
    df = pd.DataFrame(columns=['Title','Thumbsup', 'Name','Comment'])    
    for web in web_list:        
        # title
        title = web.find('yt-formatted-string',{'class': 'style-scope ytd-video-primary-info-renderer'}).getText()
        
        comments = web.find_all('div', {'id':'main', 'class': 'style-scope ytd-comment-renderer'})
        comments_names_list = []
        comments_contents_list = []
        comments_thumbsup_list = []
            
        for comment in comments:
            row = {}  
            row['Title'] = title
            # reviewer's thumbsup amount
            thumbsup = comment.find('span', {'id':'vote-count-middle', 'class':'style-scope ytd-comment-action-buttons-renderer'}).getText()
            new_thumbsup = thumbsup.strip('\n ').replace('K','*1000')
            row['Thumbsup'] = int(eval(new_thumbsup))
            # reviewer's name
            name = comment.find('span', {'class': 'style-scope ytd-comment-renderer'}).getText()        
            row['Name'] = name.strip('\n ')
            # reviewer's comment
            content = comment.find('yt-formatted-string', {'id': 'content-text'}).getText()
            row['Comment'] = content
            # append to dataframe
            df = df.append(row, ignore_index=True)
    
    return df

In [70]:
df = html_to_list(web_list)
df

,Title,Thumbsup,Name,Comment
0,Ramy Youssef: Feelings (2019) | Why Do They Ma...,2400,Kenza B.,"He touched something really deep, first he ddn..."
1,Ramy Youssef: Feelings (2019) | Why Do They Ma...,1500,LMivha,How this man got his point across is absolute ...
2,Ramy Youssef: Feelings (2019) | Why Do They Ma...,1200,Marissa Urias,"Wow, people in the comments really don't get t..."
3,Ramy Youssef: Feelings (2019) | Why Do They Ma...,909,Riaz Uddin,"This is what I tell people all the time, just ..."
4,Ramy Youssef: Feelings (2019) | Why Do They Ma...,837,Shannon dobbs,They don't ask NUNS THAT! 🙄
...,...,...,...,...
17748,Black Music Tells You Everything You Need to K...,0,357 Magnum,I'm guessing he doesn't live in a colorblind w...
17749,Black Music Tells You Everything You Need to K...,3,Erin James,Then leave my dear friend. Youve been free for...
17750,Black Music Tells You Everything You Need to K...,0,I'm black So I'm slow,We didn't create the blue's!... We didn't crea...
17751,Black Music Tells You Everything You Need to K...,2,ralph salotto,"Now that you mention it he's right, the Blues ..."


In [71]:
df.to_csv('backup_scrape_result_80.csv',index=False)

In [72]:
df.to_pickle('./pickles/scrape_result_80.pkl')

In [80]:
df_name = df.groupby(["Name"], sort=True).agg(['count']).Title.sort_values('count', ascending=False)
df_name.head(100)

,count
Name,
whitepride1004,31
Sumukh,7
Michael,6
Green Envy,6
Audrey New,6
...,...
Bi- Han,3
Lydia Waweru-Morgan,3
James Bond,3
